In [7]:
import pandas as pd
import json
import matplotlib.pyplot as plt

In [8]:
# 1. Load Data Mentah
file_path = '../data/raw/dirty_logs.txt'
with open(file_path, 'r') as f:
    content = f.read()

# File uses Python-style assignment but JSON-style null values
# Replace 'null' with 'None' so Python can interpret it
content = content.replace(': null', ': None')

local_vars = {}
exec(content, {}, local_vars)
data = local_vars['logs']

df = pd.DataFrame(data)
df.head()

,log_id,user_id,timestamp,device_platform,event_type,session_duration_sec
0,L1,U-1017,1759380000,Android,view_item,427
1,L2,guest,2025-10-10T13:40:12+00:00,google,view_category,174
2,L3,U-1046,2025-10-01T23:19:48+00:00,ios,Logout,237
3,L4,None,2025-10-05T06:25:37+00:00,web,view_item,447
4,L5,1079,1759681028,ios,Logout,162


In [9]:
# 2. Cek Overview
print("=== Info Data ===")
print(df.info())

=== Info Data ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   log_id                1000 non-null   object
 1   user_id               943 non-null    object
 2   timestamp             1000 non-null   object
 3   device_platform       1000 non-null   object
 4   event_type            957 non-null    object
 5   session_duration_sec  930 non-null    object
dtypes: object(6)
memory usage: 47.0+ KB
None


In [10]:
# 3. Cek Anomali Timestamp (Sesuai Soal)
print("\n=== Sampel Timestamp Unik ===")
print(df['timestamp'].sample(10).values)
# Temuan: Ada format int (Unix), format 'DD/MM/YYYY', dan ISO.


=== Sampel Timestamp Unik ===
['09/10/2025 21:27:31' '2025-10-11T05:34:33+00:00' 1759775401
 '11/10/2025 07:00:43' '1759644555' 1760023506 '2025-10-04T00:44:15+00:00'
 '1760457987' '2025-10-02T13:23:05+00:00' '1759960123']


In [11]:
# 4. Cek User ID bermasalah
print("\n=== User ID Aneh ===")
print(df['user_id'].unique()[:20])
# Temuan: Ada 'guest', '', dan integer.


=== User ID Aneh ===
['U-1017' 'guest' 'U-1046' None 1079 '' 'U-1001' 'U-1003' 'U-1042'
 'U-1002' 'U-1047' 'U-1045' 'U-1015' 1078 'U-1037' 1064 'U-1048' 'U-1016'
 'U-1040' 'U-1009']


In [12]:
# 5. Cek Durasi Negatif
print("\n=== Durasi Session Negatif ===")
invalid_duration = df[pd.to_numeric(df['session_duration_sec'], errors='coerce') < 0]
print(invalid_duration[['session_duration_sec']].head())


=== Durasi Session Negatif ===
   session_duration_sec
20                   -1
21                  -99
22                   -1
29                  -99
34                   -1


In [13]:
# 6. Cek Platform Inkonsisten
print("\n=== Value Counts Platform ===")
print(df['device_platform'].value_counts())
# Temuan: 'google', 'Android', 'android' harus disatukan.


=== Value Counts Platform ===
device_platform
ios        127
Android    120
web        116
iOS        115
google     114
Unknown    106
android    105
WebApp     101
Apple       96
Name: count, dtype: int64
